### Preparing Test Data

In [1]:
import pandas as pd

# Load features data
features_df = pd.read_csv('data/features.csv')

# Load stores data
stores_df = pd.read_csv('data/stores.csv')

# Load train data
train_df = pd.read_csv('data/train.csv')

# Load test data
test_df = pd.read_csv('data/test.csv')

merged_df = pd.read_csv('data/merged.csv')

In [2]:
test_df["Date"] = pd.to_datetime(test_df["Date"])
features_df["Date"] = pd.to_datetime(features_df["Date"])

In [3]:
temp_merged_df = pd.merge(test_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')
test_merged_df = pd.merge(temp_merged_df, stores_df, on='Store', how='left')
test_merged_df

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,2013-06-28,False,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,B,118221
115060,45,98,2013-07-05,False,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,B,118221
115061,45,98,2013-07-12,False,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,B,118221
115062,45,98,2013-07-19,False,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,B,118221


In [4]:
# Fill NaNs in 'CPI' with mean of CPI
test_merged_df['CPI'].fillna(test_merged_df['CPI'].mean(), inplace=True)

# Fill NaNs in 'Unemployment' with mean of Unemployment
test_merged_df['Unemployment'].fillna(test_merged_df['Unemployment'].mean(), inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_5272\480201793.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_merged_df['CPI'].fillna(test_merged_df['CPI'].mean(), inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5272\480201793.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [5]:
test_merged_df['Week'] = test_merged_df['Date'].dt.isocalendar().week
test_merged_df['Month'] = test_merged_df['Date'].dt.month
test_merged_df['Year'] = test_merged_df['Date'].dt.year
test_merged_df[["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]] = test_merged_df[["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]].fillna(0)

In [6]:
import joblib
final_ets_model = joblib.load("final_ets_model.pkl")
n_forecast_steps = len(test_merged_df["Date"].unique())
ets_predictions = final_ets_model.forecast(n_forecast_steps)

In [7]:
test_merged_df['Store_Dept_Pair'] = test_merged_df['Store'].astype(str) + '_' + test_merged_df['Dept'].astype(str)
num_pairs_per_date = test_merged_df.groupby('Date')['Store_Dept_Pair'].nunique()
num_pairs_per_date.index = pd.to_datetime(num_pairs_per_date.index)

In [8]:
ets_data = pd.concat([ets_predictions, num_pairs_per_date], axis=1)
ets_data.reset_index(inplace=True)

In [9]:
ets_data["ETS Prediction"] = ets_data[0]/ets_data["Store_Dept_Pair"]
ets_data.drop(columns = [0, "Store_Dept_Pair"], inplace=True)

In [10]:
test_merged_df["Date"] = pd.to_datetime(test_merged_df["Date"])

In [11]:
ets_data.rename(columns = {"index" : "Date"}, inplace = True)

In [12]:
new_test_df = pd.merge(test_merged_df, ets_data, on='Date', how='left')

In [13]:
import numpy as np
zero_series = pd.Series(np.zeros(len(new_test_df)))

In [14]:
new_test_df["Weekly_Sales_lag1"] = zero_series
new_test_df["Weekly_Sales_lag2"] = zero_series
new_test_df["rolling_avg_4w"] = zero_series
new_test_df["rolling_max_4w"] = zero_series
new_test_df["rolling_avg_8w"] = zero_series
new_test_df["rolling_max_8w"] = zero_series

In [15]:
ets_preds = new_test_df["ETS Prediction"]

In [16]:
new_test_df_2 = new_test_df.copy()
new_test_df_2["Weekly_Sales"] = zero_series

In [17]:
train_merged_df = pd.read_csv("data/train_merged.csv")

In [18]:
train_merged_df["Date"] = pd.to_datetime(train_merged_df["Date"])

In [19]:
new_dates = sorted(list(new_test_df["Date"].unique()))
first_eight_dates = new_dates[:8]

In [20]:
sales_dict = {}

In [21]:
for index, row in train_merged_df.iterrows():
    sales_dict[(row["Store"], row["Dept"], row["Date"])] = row["Weekly_Sales"]

In [22]:
from datetime import timedelta
def update_lag_roll_features(row):
    date = row["Date"]
    prev_eight_values = []
    for i in range(8):
        date = date - timedelta(days=7)
        try:
            sales_val = sales_dict[(row["Store"], row["Dept"], date)]
        except:
            sales_val = 0
        prev_eight_values.append(sales_val)
    row["Weekly_Sales_lag1"] = prev_eight_values[0]
    row["Weekly_Sales_lag2"] = prev_eight_values[1]
    row["rolling_avg_4w"] = sum(prev_eight_values[:4])/4
    row["rolling_avg_8w"] = sum(prev_eight_values)/8
    row["rolling_max_4w"] = max(prev_eight_values[:4])
    row["rolling_max_8w"] = max(prev_eight_values)
    return row

In [23]:
train_merged_df[(train_merged_df["Store"] == 34) & (train_merged_df["Dept"] == 39)]

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Year,Weekly_Sales_lag1,Weekly_Sales_lag2,rolling_avg_4w,rolling_max_4w,rolling_avg_8w,rolling_max_8w,Store_Dept_Pair,ETS Prediction,ML_Residuals


In [24]:
new_test_df[(train_merged_df["Store"] == 34) & (train_merged_df["Dept"] == 39)]

C:\Users\Admin\AppData\Local\Temp\ipykernel_5272\1831733134.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_test_df[(train_merged_df["Store"] == 34) & (train_merged_df["Dept"] == 39)]


,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Month,Year,Store_Dept_Pair,ETS Prediction,Weekly_Sales_lag1,Weekly_Sales_lag2,rolling_avg_4w,rolling_max_4w,rolling_avg_8w,rolling_max_8w


In [25]:
new_test_df

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Month,Year,Store_Dept_Pair,ETS Prediction,Weekly_Sales_lag1,Weekly_Sales_lag2,rolling_avg_4w,rolling_max_4w,rolling_avg_8w,rolling_max_8w
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,...,11,2012,1_1,16151.528345,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,...,11,2012,1_1,16171.278129,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,...,11,2012,1_1,15856.827771,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,...,11,2012,1_1,22685.530622,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2012-11-30,False,52.34,3.207,2460.03,0.00,3838.35,150.57,...,11,2012,1_1,17361.948941,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,2013-06-28,False,76.05,3.639,4842.29,975.03,3.00,2449.97,...,6,2013,45_98,16992.808508,0.0,0.0,0.0,0.0,0.0,0.0
115060,45,98,2013-07-05,False,77.50,3.614,9090.48,2268.58,582.74,5797.47,...,7,2013,45_98,17140.106449,0.0,0.0,0.0,0.0,0.0,0.0
115061,45,98,2013-07-12,False,79.37,3.614,3789.94,1827.31,85.72,744.84,...,7,2013,45_98,16265.830476,0.0,0.0,0.0,0.0,0.0,0.0
115062,45,98,2013-07-19,False,82.84,3.737,2961.49,1047.07,204.19,363.00,...,7,2013,45_98,16058.287782,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
new_test_df

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Month,Year,Store_Dept_Pair,ETS Prediction,Weekly_Sales_lag1,Weekly_Sales_lag2,rolling_avg_4w,rolling_max_4w,rolling_avg_8w,rolling_max_8w
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,...,11,2012,1_1,16151.528345,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,...,11,2012,1_1,16171.278129,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,...,11,2012,1_1,15856.827771,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,...,11,2012,1_1,22685.530622,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2012-11-30,False,52.34,3.207,2460.03,0.00,3838.35,150.57,...,11,2012,1_1,17361.948941,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,2013-06-28,False,76.05,3.639,4842.29,975.03,3.00,2449.97,...,6,2013,45_98,16992.808508,0.0,0.0,0.0,0.0,0.0,0.0
115060,45,98,2013-07-05,False,77.50,3.614,9090.48,2268.58,582.74,5797.47,...,7,2013,45_98,17140.106449,0.0,0.0,0.0,0.0,0.0,0.0
115061,45,98,2013-07-12,False,79.37,3.614,3789.94,1827.31,85.72,744.84,...,7,2013,45_98,16265.830476,0.0,0.0,0.0,0.0,0.0,0.0
115062,45,98,2013-07-19,False,82.84,3.737,2961.49,1047.07,204.19,363.00,...,7,2013,45_98,16058.287782,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Step 1: Preprocess the data
df = new_test_df.copy()

# Convert 'Date' to numeric features
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday
df['IsWeekend'] = df['Weekday'].isin([5, 6]).astype(int)
# df = df.drop(columns='Date')

# One-hot encode categorical columns
df = pd.get_dummies(df, columns=['Type', 'Store_Dept_Pair'], drop_first=True)

df['ETS Prediction'] = ets_preds  # base model prediction

In [28]:
df

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Store_Dept_Pair_9_87,Store_Dept_Pair_9_9,Store_Dept_Pair_9_90,Store_Dept_Pair_9_91,Store_Dept_Pair_9_92,Store_Dept_Pair_9_94,Store_Dept_Pair_9_95,Store_Dept_Pair_9_96,Store_Dept_Pair_9_98,Store_Dept_Pair_9_99
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,...,False,False,False,False,False,False,False,False,False,False
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,...,False,False,False,False,False,False,False,False,False,False
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,...,False,False,False,False,False,False,False,False,False,False
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,...,False,False,False,False,False,False,False,False,False,False
4,1,1,2012-11-30,False,52.34,3.207,2460.03,0.00,3838.35,150.57,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,2013-06-28,False,76.05,3.639,4842.29,975.03,3.00,2449.97,...,False,False,False,False,False,False,False,False,False,False
115060,45,98,2013-07-05,False,77.50,3.614,9090.48,2268.58,582.74,5797.47,...,False,False,False,False,False,False,False,False,False,False
115061,45,98,2013-07-12,False,79.37,3.614,3789.94,1827.31,85.72,744.84,...,False,False,False,False,False,False,False,False,False,False
115062,45,98,2013-07-19,False,82.84,3.737,2961.49,1047.07,204.19,363.00,...,False,False,False,False,False,False,False,False,False,False


In [29]:
len(df.columns)

3197

In [32]:
columns = joblib.load("model_features.pkl")

In [33]:
new_cols = df.columns
[x for x in new_cols if x not in columns]

['Date',
 'Store_Dept_Pair_10_99',
 'Store_Dept_Pair_18_43',
 'Store_Dept_Pair_24_43',
 'Store_Dept_Pair_25_99',
 'Store_Dept_Pair_34_39',
 'Store_Dept_Pair_36_30',
 'Store_Dept_Pair_37_29',
 'Store_Dept_Pair_42_30',
 'Store_Dept_Pair_45_39',
 'Store_Dept_Pair_5_99',
 'Store_Dept_Pair_9_99']

In [34]:
to_drop = [x for x in new_cols if x not in columns and x != 'Date']
df.drop(columns = to_drop, inplace=True)
len(df.columns)

3186

In [35]:
len([x for x in columns if x not in new_cols])

173

In [36]:
# Create a dict of all missing columns filled with zeros
to_add = [x for x in columns if x not in new_cols]
zero_data = {col: zero_series for col in to_add}

# Use pd.concat to join all at once
df = pd.concat([df, pd.DataFrame(zero_data)], axis=1)

In [37]:
len(df.columns)

3359

In [38]:
df

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Store_Dept_Pair_7_78,Store_Dept_Pair_7_99,Store_Dept_Pair_8_51,Store_Dept_Pair_8_77,Store_Dept_Pair_8_78,Store_Dept_Pair_9_51,Store_Dept_Pair_9_77,Store_Dept_Pair_9_78,Store_Dept_Pair_9_93,Store_Dept_Pair_9_97
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2012-11-30,False,52.34,3.207,2460.03,0.00,3838.35,150.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,2013-06-28,False,76.05,3.639,4842.29,975.03,3.00,2449.97,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115060,45,98,2013-07-05,False,77.50,3.614,9090.48,2268.58,582.74,5797.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115061,45,98,2013-07-12,False,79.37,3.614,3789.94,1827.31,85.72,744.84,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115062,45,98,2013-07-19,False,82.84,3.737,2961.49,1047.07,204.19,363.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
len(test_df["Date"].unique())

39

In [40]:
import joblib
model1 = joblib.load("rf_model.pkl")
model2 = joblib.load("xgb_model.pkl")

In [41]:
import warnings
from pandas.errors import PerformanceWarning

# Suppress PerformanceWarnings
warnings.simplefilter(action='ignore', category=PerformanceWarning)

In [42]:
len(sales_dict)

421570

In [43]:
len(df)

115064

In [44]:
for i in range(39):
    curr_date = new_dates[i]
    restricted_df = df[df["Date"] == curr_date]
    restricted_df = restricted_df.apply(update_lag_roll_features, axis = 1)
    date = restricted_df["Date"]
    restricted_df.drop(columns = ["Date"], inplace=True)
    restricted_df = restricted_df[columns]
    preds1 = model1.predict(restricted_df)
    preds2 = model2.predict(restricted_df)
    restricted_df["Date"] = date
    restricted_df["ml_residual"] = (preds1 + preds2)/2
    restricted_df["Weekly_Sales"] = restricted_df["ETS Prediction"] + restricted_df["ml_residual"]
    restricted_df = restricted_df.copy()
    for index, row in restricted_df.iterrows():
        sales_dict[(row["Store"], row["Dept"], row["Date"])] = row["Weekly_Sales"]

In [45]:
len(sales_dict)

536634

In [46]:
test_df

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


In [47]:
def update_weekly_sales(row):
    row["Weekly_Sales"] = sales_dict[(row["Store"], row["Dept"], pd.to_datetime(row["Date"]))]
    return row
test_df = test_df.apply(update_weekly_sales, axis = 1)
test_df

,Store,Dept,Date,IsHoliday,Weekly_Sales
0,1,1,2012-11-02,False,34566.971954
1,1,1,2012-11-09,False,35897.214207
2,1,1,2012-11-16,False,33536.574195
3,1,1,2012-11-23,True,56573.033078
4,1,1,2012-11-30,False,43266.214387
...,...,...,...,...,...
115059,45,98,2013-06-28,False,4849.590078
115060,45,98,2013-07-05,False,5122.644326
115061,45,98,2013-07-12,False,5255.335080
115062,45,98,2013-07-19,False,5069.009258


In [48]:
test_df.to_csv("data/predictions.csv", index=False)